# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [2]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [4]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {'table':["users", "orders", "products"],
        'definition': ["Stores user information such as id, name, email",
        "Stores order details such as order_id, user_id, total_price",
        "Stores product information such as product_id, name, price"]}
df = pd.DataFrame(data)
print(df)

      table                                         definition
0     users    Stores user information such as id, name, email
1    orders  Stores order details such as order_id, user_id...
2  products  Stores product information such as product_id,...


In [5]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [6]:
print(text_tables)

users: Stores user information such as id, name, email
orders: Stores order details such as order_id, user_id, total_price
products: Stores product information such as product_id, name, price


In [9]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""

user_query = "Find the total price of all orders for a given user."


In [10]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question=user_query)

In [11]:
print(return_OAI(pqt1))

{
    "tables": ["users", "orders", "products"]
}


In [12]:
user_query = "Find the average price of products sold in the last month."
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question=user_query)

In [13]:
print(return_OAI(pqt3))

{
    "tables": ["orders", "products"]
}


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [17]:
# User wants to know all users who placed orders
user_query = "List all users who have placed at least one order."
pqt2 = prompt_question_tables.format(
    tables=text_tables,
    question=user_query
)
print(return_OAI(pqt2))

```json
{
    "tables": ["users", "orders"]
}
```


In [18]:
# User wants product info
user_query = "Show me the names and prices of all products."
pqt4 = prompt_question_tables.format(
    tables=text_tables,
    question=user_query
)
print(return_OAI(pqt4))

```json
{
    "tables": ["products"]
}
```


In [ ]:
# User wants to find inactive users
user_query = "Find users who have not placed any orders."
pqt6 = prompt_question_tables.format(
    tables=text_tables,
    question=user_query
)
print(return_OAI(pqt6))

{
    "tables": ["users", "orders"]
}


Report on SQL Prompting with GPT




In this lab, I tested how GPT can turn a normal question into SQL queries by using the names of tables. The idea was to see if different prompts would change the results, and to check when GPT gave good answers or when it made mistakes.

What Worked Well

When I gave GPT clear instructions with the right table names, it usually worked fine. For example, when I asked about the average price of products sold last month, it picked the right tables like users, orders, and products. The answers made sense, and the format was easy to read.

What Didn’t Work Well

Some prompts did not go as planned. If the question was vague or I didn’t explain the tables properly, GPT sometimes made up things. For example, it returned tables like “employees,” “salary,” or “studies” that were never part of the schema. That showed me GPT can “hallucinate” and invent stuff when it is unsure.

Another small problem was that the output format was not always the same. Sometimes GPT gave plain JSON, and other times it wrapped it inside code blocks. This is not a big deal, but it could be annoying if I wanted to parse the results automatically.

What I Learned

I learned that the way I write the prompt makes a big difference. If I am very clear and give full information, GPT does much better. If I leave gaps, GPT tries to fill them but is often wrong. I also learned that I can’t just trust the first answer—it’s important to check and validate the output.

Overall, this lab showed me that GPT is a useful tool for generating SQL, but it needs careful prompts and human oversight. The main lesson is that good prompts = good results, while unclear prompts = hallucinations or mistakes.